# Creating a climate index API

In order to know which destinations have (or will have) the best weather on a given date we will be using historical data from all of Spain's weather stations, as well as packaging that information into an API that will allow us to remotely perform queries.

Since all information from 291 weather stations are stored in individual *csv* files, let's import the first one and see how can we coax the necessary data out of it.

# Transforming historical data into a weather index

*Indices are particularly valuable because they allow the integrated effects of a range of climatic variables to be quantified, facilitating an interpretation and rating of climatic conditions at a destination. Another advantage of indices is that they enable the climate of tourism destinations to be objectively compared and are therefore a convenient and more conceptually sound means to assess possible impacts of climate change on the distribution of climatic resources worldwide.*

                  - An Inter-Comparison of the Holiday Climate Index (HCI) and the Tourism Climate Index (TCI) in Europe
                    https://www.mdpi.com/2073-4433/7/6/80/htm
                    
                    
                    
                    
Our aim is to condense all meteorological data (wind, temperature, hours of daylight, rain...) into a single numnber (an index) that can be used to easily compare one destination with another. The objective is to create a dataframe where every row is a different destination and each column holds the index for each week of the year.

We can't use the **TCI** as-is because we lack some of the data (humidity and cloud cover %), so we will have to make an index of our own. From now on we will call it simply the **CWI** (**C**ycling **W**eather **I**ndex).

## Creating our **CWI**

While it would be nice to have all weather data for every town in Spain, we have to make do with the available datasets.

Of the present variables in the official meteo datasets, the following variables are of use:


- Date (to group values per week).
- Average temperature.
- Rain (mm).
- Average wind speed.


The hours of sunlight aren't really meaningful because we're crafting this custom index to be used only on Spain, which has a single timezone and the latitude change isn't that great.




## Importing our data

The first step is importing our data. Every weather station's data is contained in a different *csv* file, we'll import the first one to see how data is structured.

In [1]:
import pandas as pd

In [10]:
#Importing our dataframe.

df = pd.read_csv('meteo_test.csv', sep=';')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17390 entries, 0 to 17389
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FECHA          17390 non-null  object 
 1   INDICATIVO     17390 non-null  object 
 2   NOMBRE         17390 non-null  object 
 3   PROVINCIA      17390 non-null  object 
 4   ALTITUD        17390 non-null  int64  
 5   TMEDIA         16212 non-null  float64
 6   PRECIPITACION  16081 non-null  object 
 7   TMIN           16212 non-null  float64
 8   HORATMIN       16191 non-null  object 
 9   TMAX           16231 non-null  float64
 10  HORATMAX       16218 non-null  object 
 11  DIR            15588 non-null  float64
 12  VELMEDIA       16233 non-null  float64
 13  RACHA          15569 non-null  float64
 14  HORARACHA      15539 non-null  object 
 15  SOL            10102 non-null  float64
 16  PRESMAX        8928 non-null   float64
 17  HORAPRESMAX    8899 non-null   object 
 18  PRESMI

In [16]:
df.head()

,FECHA,INDICATIVO,NOMBRE,PROVINCIA,ALTITUD,TMEDIA,PRECIPITACION,TMIN,HORATMIN,TMAX,HORATMAX,DIR,VELMEDIA,RACHA,HORARACHA,SOL,PRESMAX,HORAPRESMAX,PRESMIN,HORAPRESMIN
0,1968-03-01,0002I,VANDELL�S,TARRAGONA,32,8.9,21.0,6.6,03:00,11.2,18:00,5.0,1.9,6.7,10:55,0.0,NaN,NaN,NaN,NaN
1,1968-03-02,0002I,VANDELL�S,TARRAGONA,32,10.9,0.0,6.0,07:10,15.8,15:00,32.0,1.1,18.6,23:40,8.6,NaN,NaN,NaN,NaN
2,1968-03-03,0002I,VANDELL�S,TARRAGONA,32,NaN,0.0,NaN,NaN,NaN,NaN,32.0,6.1,19.2,00:55,8.6,NaN,NaN,NaN,NaN
3,1968-03-04,0002I,VANDELL�S,TARRAGONA,32,10.9,6.5,7.8,05:00,14.0,11:40,32.0,3.9,8.1,06:10,8.4,NaN,NaN,NaN,NaN
4,1968-03-05,0002I,VANDELL�S,TARRAGONA,32,NaN,0.0,NaN,NaN,NaN,NaN,32.0,3.6,13.1,23:40,2.5,NaN,NaN,NaN,NaN


## Using time series to get the average values

Since we want to calculate the **TCI** on a weekly basis it makes sense to compact our data with the same granularity. For this purpose we will be using time series.

In [17]:
#Converting 'FECHA' to Datetime format.

df["FECHA"] = df['FECHA'].apply(pd.to_datetime)

In [29]:
#Checking the result.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17390 entries, 0 to 17389
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   FECHA          17390 non-null  datetime64[ns]
 1   INDICATIVO     17390 non-null  object        
 2   NOMBRE         17390 non-null  object        
 3   PROVINCIA      17390 non-null  object        
 4   ALTITUD        17390 non-null  int64         
 5   TMEDIA         16212 non-null  float64       
 6   PRECIPITACION  16081 non-null  object        
 7   TMIN           16212 non-null  float64       
 8   HORATMIN       16191 non-null  object        
 9   TMAX           16231 non-null  float64       
 10  HORATMAX       16218 non-null  object        
 11  DIR            15588 non-null  float64       
 12  VELMEDIA       16233 non-null  float64       
 13  RACHA          15569 non-null  float64       
 14  HORARACHA      15539 non-null  object        
 15  SOL            1010

Since the climate is changing it would be foolish to use all data for our forecast. The last 5 years will suffice.

In [51]:
#Creating a dataframe with data from the last 5 years.

filtered = df.loc[(df['FECHA'] >= '2016-01-01') & (df['FECHA'] < '2021-01-01')]

#Resetting the index.

filtered.reset_index(drop=True, inplace=True)

In [52]:
filtered.head()

,FECHA,INDICATIVO,NOMBRE,PROVINCIA,ALTITUD,TMEDIA,PRECIPITACION,TMIN,HORATMIN,TMAX,HORATMAX,DIR,VELMEDIA,RACHA,HORARACHA,SOL,PRESMAX,HORAPRESMAX,PRESMIN,HORAPRESMIN
0,2016-01-01,0002I,VANDELL�S,TARRAGONA,32,13.6,0.1,10.8,04:40,16.4,13:00,12.0,0.6,3.6,00:10,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,0002I,VANDELL�S,TARRAGONA,32,13.6,0.0,10.7,21:10,16.4,14:20,99.0,2.8,13.1,10:30,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,0002I,VANDELL�S,TARRAGONA,32,12.1,0.0,9.6,23:00,14.6,11:50,24.0,1.4,8.6,09:30,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,0002I,VANDELL�S,TARRAGONA,32,12.7,0.3,9.4,01:00,16.0,04:40,33.0,1.7,14.2,20:40,NaN,NaN,NaN,NaN,NaN
4,2016-01-05,0002I,VANDELL�S,TARRAGONA,32,12.0,0.0,9.7,23:59,14.4,13:30,30.0,3.1,17.5,03:10,NaN,NaN,NaN,NaN,NaN


In [53]:
#We don't have data for every single day, but this will be enough.

filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1291 entries, 0 to 1290
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   FECHA          1291 non-null   datetime64[ns]
 1   INDICATIVO     1291 non-null   object        
 2   NOMBRE         1291 non-null   object        
 3   PROVINCIA      1291 non-null   object        
 4   ALTITUD        1291 non-null   int64         
 5   TMEDIA         1276 non-null   float64       
 6   PRECIPITACION  1126 non-null   object        
 7   TMIN           1276 non-null   float64       
 8   HORATMIN       1274 non-null   object        
 9   TMAX           1276 non-null   float64       
 10  HORATMAX       1274 non-null   object        
 11  DIR            1236 non-null   float64       
 12  VELMEDIA       1241 non-null   float64       
 13  RACHA          1236 non-null   float64       
 14  HORARACHA      1236 non-null   object        
 15  SOL            0 non-

Now that we have a dataset of the last 5 years it's time to calculate the weekly averages.

In [59]:
#Using datetime to add the week number as a new column.

filtered['week'] = None
weekNumber = filtered['FECHA'].dt.week.tolist()
filtered['week'] = weekNumber

<ipython-input-59-39abab0bf700>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['week'] = None
<ipython-input-59-39abab0bf700>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  weekNumber = filtered['FECHA'].dt.week.tolist()
<ipython-input-59-39abab0bf700>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['week'] = weekNumber


In [60]:
filtered.head()

,FECHA,INDICATIVO,NOMBRE,PROVINCIA,ALTITUD,TMEDIA,PRECIPITACION,TMIN,HORATMIN,TMAX,...,DIR,VELMEDIA,RACHA,HORARACHA,SOL,PRESMAX,HORAPRESMAX,PRESMIN,HORAPRESMIN,week
0,2016-01-01,0002I,VANDELL�S,TARRAGONA,32,13.6,0.1,10.8,04:40,16.4,...,12.0,0.6,3.6,00:10,NaN,NaN,NaN,NaN,NaN,53
1,2016-01-02,0002I,VANDELL�S,TARRAGONA,32,13.6,0.0,10.7,21:10,16.4,...,99.0,2.8,13.1,10:30,NaN,NaN,NaN,NaN,NaN,53
2,2016-01-03,0002I,VANDELL�S,TARRAGONA,32,12.1,0.0,9.6,23:00,14.6,...,24.0,1.4,8.6,09:30,NaN,NaN,NaN,NaN,NaN,53
3,2016-01-04,0002I,VANDELL�S,TARRAGONA,32,12.7,0.3,9.4,01:00,16.0,...,33.0,1.7,14.2,20:40,NaN,NaN,NaN,NaN,NaN,1
4,2016-01-05,0002I,VANDELL�S,TARRAGONA,32,12.0,0.0,9.7,23:59,14.4,...,30.0,3.1,17.5,03:10,NaN,NaN,NaN,NaN,NaN,1


In [62]:
#Creating a dataframe with the weekly mean values.

df_mean = filtered.groupby("week").mean()

In [63]:
df_mean.head()

,ALTITUD,TMEDIA,TMIN,TMAX,DIR,VELMEDIA,RACHA,SOL,PRESMAX,PRESMIN
week,,,,,,,,,,
1,32,11.708571,8.011429,15.417143,28.457143,2.780000,14.708571,NaN,NaN,NaN
2,32,10.165714,6.728571,13.602857,30.885714,3.382857,16.685714,NaN,NaN,NaN
3,32,10.502857,6.940000,14.082857,30.285714,2.831429,14.020000,NaN,NaN,NaN
4,32,11.206667,8.053333,14.326667,33.300000,3.110000,15.663333,NaN,NaN,NaN
5,32,10.964286,7.253571,14.664286,28.821429,2.600000,14.267857,NaN,NaN,NaN
